(sec-dask-gpu)=
# GPU

GPUs and other heterogeneous accelerators are widely utilized to accelerate deep learning. The Dask community, in collaboration with NVIDIA, has provided a GPU-based toolkit for data science to expedite a variety of tasks.

## Dask GPU Cluster

[Dask-CUDA](https://docs.rapids.ai/api/dask-cuda/stable/) is an extension to `dask.distributed` that enables detecting and managing GPU devices. Users should instal Dask-CUDA via `pip install dask-cuda`. Like `dask.distributed` discussed in {numref}`sec-dask-distributed`, Dask-CUDA offers a `LocalCUDACluster` for a single machine. The `LocalCUDACluster` automatically detects and registers the multiple GPUs on the computing node, assigning a certain number of CPU cores to each GPU. For instance, in an environment equipped with 4 GPUs, initiating a single-machine Dask cluster will launch 4 Dask workers, with each worker allocated one GPU.

In [1]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)
client

/fs/fast/u20200002/envs/dispy/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37111 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:46657' processes=4 threads=4, memory=90.00 GiB>

We can launch a Dask GPU cluster by first launching the Dask Scheduler.

```
dask scheduler
```

Subsequently, launch a Dask GPU Worker on each GPU node. Thus, we have a Dask GPU cluster.

```
dask cuda worker tcp://scheduler:8786
```

In [4]:
client = Client("10.0.0.3:8786")
client

<Client: 'tcp://10.0.0.3:8786' processes=8 threads=8, memory=180.00 GiB>

:::{note}
Dask-CUDA only discovers and registers these GPUs, but it is incapable of isolating GPUs. Other non-Dask tasks can still preempt these GPUs. Isolation of GPU resources should be achieved by leveraging container technologies such as Kubernetes. 
:::

## GPU Task

Not all tasks can be accelerated by GPUs. GPUs are mainly used to expedite computationally intensive tasks, such as machine learning and deep learning. At present, the frameworks supported by Dask on GPUs include:

* Scaling [CuPy](https://cupy.dev/) to a GPU cluster.
* Scaling [Dask-cudf](https://docs.rapids.ai/api/dask-cudf/stable/) DataFrame to a GPU cluster.

:::{note}
When utilizing NVIDIA's GPUs, it is necessary to append the CUDA directory to the `PATH` and `LD_LIBRARY_PATH` environment variables, as CuPy and cuDF depend on NVIDIA's GPU libraries.
:::

### Example: SVD

The following code performs Singular Value Decomposition (SVD) on a GPU, which is a task well-suited for GPU acceleration. By setting `dask.config.set({"array.backend": "cupy"})`, the execution backend for Dask Array can be changed to CuPy on the GPU.

```python
import cupy
import dask
import dask.array as da

dask.config.set({"array.backend": "cupy"})
rs = dask.array.random.RandomState(RandomState=cupy.random.RandomState)
x = rs.random((10000, 1000), chunks=(1000, 1000))
u, s, v = dask.array.linalg.svd(x)
```